In [3]:
from functions import *
path = '..//data//HighD//mat//'
full = loadmat(path+'highD_full.mat')['full']
full = fill_nan(full)
# plt.matshow(full, cmap='jet_r')

r_list = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
i_list = list(range(10))

np.random.seed(0)
for r in r_list:
    for i in i_list:
        maxiter = 1500
        observed = loadmat(path+f'highD_{r}_{i}.mat')['s']
        mask = ~np.isnan(observed)
        n = min(max(int((~np.isnan(observed)).sum() * 0.02), 100), 500)  # number of inducing points
        rx = np.random.uniform(low=0, high=full.shape[1], size=(n,1))
        ry = np.random.uniform(low=0, high=full.shape[0], size=(n,1))
        # plt.matshow(full, cmap='jet_r')

        train_X = np.where(mask == 1)
        train_Y = observed[train_X].reshape(-1, 1)
        train_X = np.concatenate([x.reshape([-1, 1]) for x in train_X], axis = 1)
        train_X = train_X.astype(np.float64)

        mean_Y = np.mean(train_Y)
        std_Y = np.std(train_Y)
        train_Y  = (train_Y - mean_Y) / std_Y  # standardize
        vmax = full.max()
        vmin = full.min()

#         Z = np.random.permutation(train_X)[:n, :]  # inducing inputs
        Z = np.concatenate([ry, rx], axis=1).astype(np.float64)  # inducing inputs
        kernel = Rational_Quadratic  # Matern52
      #   kernel = gpflow.kernels.Matern52(lengthscales=[150.0, 13.0], variance=0.2)
        kernel = Directional_Kernel(kernel, lengthscales=[60.0, 13.0], theta=0.10, variance=0.2, alpha=10.0)
        model = gpflow.models.SGPR(data=(train_X, train_Y), kernel=kernel, mean_function=None,
                                   inducing_variable=Z, noise_variance=0.3)
        # model.training_loss()
        # print_summary(model)

        # define monitor
        test_name = f"HighD_r{r}_i{i}_{model.kernel.name}_N{n}_maxiter{maxiter}"
        # log_dir_compiled = "logs/fit/" + test_name + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        # model_task = ModelToTensorBoard(log_dir_compiled, model)
        # lml_task = ScalarToTensorBoard(
        #     log_dir_compiled, lambda: model.training_loss(), "training_objective")
        # Note that the `ImageToTensorBoard` task cannot be compiled, and is omitted from the monitoring
        # monitor = Monitor(MonitorTaskGroup(lml_task))
        name = f'r{r}_i{i}_Directional_RQkernel_N{n}_maxiter1500'
        time0 = time.time()
        # print(f'Start training {test_name} at', time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(time0)))
        opt = gpflow.optimizers.Scipy()
        opt_logs = opt.minimize(model.training_loss, model.trainable_variables, options=dict(maxiter=maxiter),
                                # step_callback=monitor
                                )
        # print(f'End training {test_name} at', time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(time.time())))

        # Examine the results
        test_X = np.where(full >= 0)
        test_X = np.concatenate([x.reshape([-1, 1]) for x in test_X], axis=1)
        test_X = test_X.astype(np.float64)
        test_Y, Var_Y = model.predict_y(test_X)

        predicted_Y = test_Y.numpy().reshape(full.shape) * std_Y + mean_Y
        Var_Y = Var_Y.numpy().reshape(full.shape)*std_Y*std_Y
        predicted_Y2 = predicted_Y.copy()  # The prediction with observed location unchanged
        predicted_Y2[mask] = observed[mask]
        predicted_Y2[predicted_Y2<0] = 0

        train_mae = mae(predicted_Y[mask], full[mask])
        train_rmse = rmse(predicted_Y[mask], full[mask])
        test_mae = mae(predicted_Y, full)
        test_rmse = rmse(predicted_Y, full)
        test_mae2 = mae(predicted_Y2, full)
        test_rmse2 = rmse(predicted_Y2, full)

        print(f'{test_name}. theta: {model.kernel.theta.numpy():.3f}, '
              f'lengthscale: {model.kernel.lengthscale.numpy()}, alpha: {model.kernel.alpha.numpy()}, time: {time.time()-time0:.3f}')
        print(f'Train mae: {train_mae:.3f}, rmse: {train_rmse:.3f}. Test mae: {test_mae:.3f}, rmse: {test_rmse:.3f}. '
              f'Test mae2: {test_mae2:.3f}, rmse2: {test_rmse2:.3f}.')

        lengthscale = model.kernel.lengthscale.numpy()
        test_name = test_name + f'_mae{test_mae:.3f}_rmse{test_rmse:.3f}' + \
                    f'theta{model.kernel.theta.numpy():.3f}_lengthscale{lengthscale[0]:.3f}_{lengthscale[1]:.3f}'

        savemat(f'./{name}.mat', {'pre_mean': predicted_Y, 'pre_var': Var_Y}, do_compression=True)

HighD_r0.05_i0_Directional_Rational_Quadratic_N100_maxiter1500. theta: 0.062, lengthscale: [49.56864222  7.55792383], alpha: 0.5216641031919105, time: 17.576
Train mae: 1.414, rmse: 1.832. Test mae: 4.981, rmse: 6.827. Test mae2: 4.943, rmse2: 6.818.
HighD_r0.05_i1_Directional_Rational_Quadratic_N100_maxiter1500. theta: 0.099, lengthscale: [99.18519651 15.74716586], alpha: 0.09654010364558323, time: 11.412
Train mae: 1.434, rmse: 1.893. Test mae: 4.465, rmse: 6.471. Test mae2: 4.427, rmse2: 6.461.
HighD_r0.05_i2_Directional_Rational_Quadratic_N100_maxiter1500. theta: 0.184, lengthscale: [37.8754924   7.04921763], alpha: 3.051480743144948, time: 11.518
Train mae: 1.589, rmse: 2.101. Test mae: 5.596, rmse: 7.837. Test mae2: 5.546, rmse2: 7.823.
HighD_r0.05_i3_Directional_Rational_Quadratic_N100_maxiter1500. theta: 0.043, lengthscale: [80.03626636 14.81233117], alpha: 0.08060075348788752, time: 11.071
Train mae: 1.245, rmse: 1.723. Test mae: 5.546, rmse: 8.323. Test mae2: 5.522, rmse2: 8.

path = '/kaggle/input/ngsim-speedfield/'
full = np.load(path+'speed_grid_full.npy')
full = fill_nan(full)
plt.matshow(full, cmap='jet_r')

# Load and save the results of the ARD kernel in ngsim data
r_list = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
i_list = list(range(10))
np.random.seed(0)

for r in [0.1]:
    for i in i_list:
        maxiter = 1500
        observed = np.load(path+f'speed_grid_{r}_{i}.npy')
        mask = ~np.isnan(observed)
        n = min(max(int((~np.isnan(observed)).sum() * 0.02), 100), 500)  # number of inducing points
        rx = np.random.uniform(low=0, high=full.shape[1], size=(n,1))
        ry = np.random.uniform(low=0, high=full.shape[0], size=(n,1))
        # plt.matshow(full, cmap='jet_r')

        train_X = np.where(mask == 1)
        train_Y = observed[train_X].reshape(-1, 1)
        train_X = np.concatenate([x.reshape([-1, 1]) for x in train_X], axis = 1)
        train_X = train_X.astype(np.float64)

        mean_Y = np.mean(train_Y)
        std_Y = np.std(train_Y)
        train_Y  = (train_Y - mean_Y) / std_Y  # standardize
        vmax = full.max()
        vmin = full.min()

#         Z = np.random.permutation(train_X)[:n, :]  # inducing inputs
        Z = np.concatenate([ry, rx], axis=1).astype(np.float64)  # inducing inputs
        kernel = gpflow.kernels.SquaredExponential(lengthscales=[5.0, 10.0])
        model = gpflow.models.SGPR(data=(train_X, train_Y), kernel=kernel, mean_function=None,
                                   inducing_variable=Z, noise_variance=0.02)
#         time0 = time.time()
#         opt = gpflow.optimizers.Scipy()
#         opt_logs = opt.minimize(model.training_loss, model.trainable_variables, options=dict(maxiter=maxiter))
        name = f"r{r}_i{i}_squared_exponential_N{n}_maxiter1500.pkl"
        params = load_pickle(f'/kaggle/input/model-param/{name}')
        gpflow.utilities.multiple_assign(model, params)
        # define monitor
#         test_name = f"r{r}_i{i}_{model.kernel.name}_N{n}_maxiter{maxiter}"

        # Examine the results
        test_X = np.where(full >= 0)
        test_X = np.concatenate([x.reshape([-1, 1]) for x in test_X], axis=1)
        test_X = test_X.astype(np.float64)
        time0 = time.time()
        test_Y, Var_Y = model.predict_y(test_X)

        predicted_Y = test_Y.numpy().reshape(full.shape) * std_Y + mean_Y
        Var_Y = Var_Y*std_Y*std_Y
        predicted_Y2 = predicted_Y.copy()  # The prediction with observed location unchanged
        predicted_Y2[mask] = observed[mask]

        train_mae = mae(predicted_Y[mask], full[mask])
        train_rmse = rmse(predicted_Y[mask], full[mask])
        test_mae = mae(predicted_Y, full)
        test_rmse = rmse(predicted_Y, full)
        test_mae2 = mae(predicted_Y2, full)
        test_rmse2 = rmse(predicted_Y2, full)
        
#         print(f'{test_name}. n:{n}, time: {time.time()-time0:.3f}, likelihood: {model.likelihood.variance.numpy()}')
        print(f'Train mae: {train_mae:.3f}, rmse: {train_rmse:.3f}. Test mae: {test_mae:.3f}, rmse: {test_rmse:.3f}. '
              f'Test mae2: {test_mae2:.3f}, rmse2: {test_rmse2:.3f}. Time: {time.time()-time0:.3f}')

        # Save the model
#         save_pickle(gpflow.utilities.parameter_dict(model), f'./{test_name}.pkl')
        savemat(f'./{name}.mat', {'pre_mean': predicted_Y, 'pre_var': Var_Y}, do_compression=True)

# Save and load the result in rotated martern kernel in ngsim dataset
r_list = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
i_list = list(range(10))
np.random.seed(0)
for r in r_list:
    for i in i_list:
        maxiter = 1500
        observed = np.load(path+f'speed_grid_{r}_{i}.npy')
        mask = ~np.isnan(observed)
        n = min(max(int((~np.isnan(observed)).sum() * 0.02), 100), 500)  # number of inducing points
        rx = np.random.uniform(low=0, high=full.shape[1], size=(n,1))
        ry = np.random.uniform(low=0, high=full.shape[0], size=(n,1))
        # plt.matshow(full, cmap='jet_r')
        
        train_X = np.where(mask == 1)
        train_Y = observed[train_X].reshape(-1, 1)
        train_X = np.concatenate([x.reshape([-1, 1]) for x in train_X], axis = 1)
        train_X = train_X.astype(np.float64)

        mean_Y = np.mean(train_Y)
        std_Y = np.std(train_Y)
        train_Y  = (train_Y - mean_Y) / std_Y  # standardize
        vmax = full.max()
        vmin = full.min()

#         Z = np.random.permutation(train_X)[:n, :]  # inducing inputs
        Z = np.concatenate([ry, rx], axis=1).astype(np.float64)  # inducing inputs
        kernel = Directional_Kernel(Matern52, lengthscales=[150, 8], theta=0.1, variance=0.2)
        model = gpflow.models.SGPR(data=(train_X, train_Y), kernel=kernel, mean_function=None,
                                   inducing_variable=Z, noise_variance=0.02)
        name = f'r{r}_i{i}_Directional_Matern52_N{n}_maxiter1500'
        params = load_pickle(f'/kaggle/input/model-param/{name}.pkl')
        gpflow.utilities.multiple_assign(model, params)
#         opt = gpflow.optimizers.Scipy()
#         opt_logs = opt.minimize(model.training_loss, model.trainable_variables, options=dict(maxiter=maxiter))
                
        # define monitor
#         test_name = f"r{r}_i{i}_{model.kernel.name}_N{n}_maxiter{maxiter}"

        # Examine the results
        test_X = np.where(full >= 0)
        test_X = np.concatenate([x.reshape([-1, 1]) for x in test_X], axis=1)
        test_X = test_X.astype(np.float64)
        time0 = time.time()
        test_Y, Var_Y = model.predict_y(test_X)

        predicted_Y = test_Y.numpy().reshape(full.shape) * std_Y + mean_Y
        Var_Y = Var_Y.numpy().reshape(full.shape)*std_Y*std_Y
        predicted_Y2 = predicted_Y.copy()  # The prediction with observed location unchanged
        predicted_Y2[mask] = observed[mask]
        predicted_Y2[predicted_Y2<0] = 0

        train_mae = mae(predicted_Y[mask], full[mask])
        train_rmse = rmse(predicted_Y[mask], full[mask])
        test_mae = mae(predicted_Y, full)
        test_rmse = rmse(predicted_Y, full)
        test_mae2 = mae(predicted_Y2, full)
        test_rmse2 = rmse(predicted_Y2, full)

#         print(f'{test_name}. theta: {model.kernel.theta.numpy():.3f}, n:{n},'
#               f'lengthscale: {model.kernel.lengthscale.numpy()}, likelihood: {model.likelihood.variance.numpy()}, time: {time.time()-time0:.3f}')
        print(f'Train mae: {train_mae:.3f}, rmse: {train_rmse:.3f}. Test mae: {test_mae:.3f}, rmse: {test_rmse:.3f}. '
              f'Test mae2: {test_mae2:.3f}, rmse2: {test_rmse2:.3f}. Time: {time.time()-time0:.3f}')

        # Save the model
#         save_pickle(gpflow.utilities.parameter_dict(model), f'./{test_name}.pkl')
        savemat(f'./{name}.mat', {'pre_mean': predicted_Y, 'pre_var': Var_Y}, do_compression=True)
    

import os
import zipfile

folder_path = "/kaggle/working/"  # replace with your folder path
zip_path = "/kaggle/working/archive.zip"  # replace with desired archive path

file_list = os.listdir(folder_path)
with zipfile.ZipFile(zip_path, "w") as zip_file:
    for filename in file_list:
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            zip_file.write(file_path, filename)


# Using the rotated kernel
r_list = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
i_list = list(range(10))
np.random.seed(0)
for r in [0.05]:
    for i in i_list:
        maxiter = 1500
        observed = np.load(path+f'speed_grid_{r}_{i}.npy')
        mask = ~np.isnan(observed)
        n = min(max(int((~np.isnan(observed)).sum() * 0.02), 100), 500)  # number of inducing points
        rx = np.random.uniform(low=0, high=full.shape[1], size=(n,1))
        ry = np.random.uniform(low=0, high=full.shape[0], size=(n,1))
        # plt.matshow(full, cmap='jet_r')

        train_X = np.where(mask == 1)
        train_Y = observed[train_X].reshape(-1, 1)
        train_X = np.concatenate([x.reshape([-1, 1]) for x in train_X], axis = 1)
        train_X = train_X.astype(np.float64)

        mean_Y = np.mean(train_Y)
        std_Y = np.std(train_Y)
        train_Y  = (train_Y - mean_Y) / std_Y  # standardize
        vmax = full.max()
        vmin = full.min()

#         Z = np.random.permutation(train_X)[:n, :]  # inducing inputs
        Z = np.concatenate([ry, rx], axis=1).astype(np.float64)  # inducing inputs
        kernel = Directional_Kernel(Matern52, lengthscales=[150, 8], theta=0.1, variance=0.2)
        model = gpflow.models.SGPR(data=(train_X, train_Y), kernel=kernel, mean_function=None,
                                   inducing_variable=Z, noise_variance=0.02)
        time0 = time.time()
        opt = gpflow.optimizers.Scipy()
        opt_logs = opt.minimize(model.training_loss, model.trainable_variables, options=dict(maxiter=maxiter))
                
        # define monitor
        test_name = f"r{r}_i{i}_{model.kernel.name}_N{n}_maxiter{maxiter}"

        # Examine the results
        test_X = np.where(full >= 0)
        test_X = np.concatenate([x.reshape([-1, 1]) for x in test_X], axis=1)
        test_X = test_X.astype(np.float64)
        test_Y = model.predict_y(test_X)[0]

        predicted_Y = test_Y.numpy().reshape(full.shape) * std_Y + mean_Y
        predicted_Y2 = predicted_Y.copy()  # The prediction with observed location unchanged
        predicted_Y2[mask] = observed[mask]

        train_mae = mae(predicted_Y[mask], full[mask])
        train_rmse = rmse(predicted_Y[mask], full[mask])
        test_mae = mae(predicted_Y, full)
        test_rmse = rmse(predicted_Y, full)
        test_mae2 = mae(predicted_Y2, full)
        test_rmse2 = rmse(predicted_Y2, full)

        print(f'{test_name}. theta: {model.kernel.theta.numpy():.3f}, n:{n},'
              f'lengthscale: {model.kernel.lengthscale.numpy()}, likelihood: {model.likelihood.variance.numpy()}, time: {time.time()-time0:.3f}')
        print(f'Train mae: {train_mae:.3f}, rmse: {train_rmse:.3f}. Test mae: {test_mae:.3f}, rmse: {test_rmse:.3f}. '
              f'Test mae2: {test_mae2:.3f}, rmse2: {test_rmse2:.3f}.')

        # Save the model
        save_pickle(gpflow.utilities.parameter_dict(model), f'./{test_name}.pkl')
        save_pickle(predicted_Y)

# Using the highD dataset for co-kriging

# Using the highD dataset for co-kriging
from scipy.io import loadmat, savemat
path = '/kaggle/input/ngsim-speedfield/' 
full3 = loadmat(path+'highD_r3_full.mat')['full_speed']
full3 = fill_nan(full3)
full4 = loadmat(path+'highD_r4_full.mat')['full']
full = [full3, full4]
vmax = max([f.max() for f in full])
vmin = min([f.min() for f in full])

fig, axes = plt.subplots(2, 1, figsize=(5, 6), sharex=True, sharey=True)
fig.subplots_adjust(hspace=0.1)
for i, ax in enumerate(axes):
    ax.matshow(full[i], cmap='jet_r')
    ax.set_title(f'Lane={i+2}')
axes[-1].xaxis.set_ticks_position('bottom')

data3 = loadmat(path+'highD_r0.05_lane3_seed0.mat')['s']
data4 = loadmat(path+'highD_r0.05_lane4_seed0.mat')['s']
data = [data3, data4]
od = len(data)  # number of output dimensions
fig, axes = plt.subplots(2, 1, figsize=(5, 6), sharex=True, sharey=True)
fig.subplots_adjust(hspace=0.1)
for i, ax in enumerate(axes):
    ax.matshow(data[i], cmap='jet_r')
    ax.set_title(f'Lane={i+2}')
axes[-1].xaxis.set_ticks_position('bottom')

# ICM
mask = [~np.isnan(d) for d in data]
train_X = [np.where(mask[i] == 1) for i in range(od)]
train_Y = [data[i][train_X[i]].reshape(-1, 1) for i in range(od)]
n_per_lane = [train_Y[i].shape[0] for i in range(od)]
train_X = [np.concatenate([x.reshape([-1, 1]) for x in train_X[i]], axis = 1) for i in range(od)]
train_X_aug = np.vstack(tuple(np.hstack((train_X[i], i*np.ones((n_per_lane[i], 1)))) for i in range(od)))

train_Y_aug = np.vstack(tuple(np.hstack((train_Y[i], i*np.ones((n_per_lane[i], 1)))) for i in range(od)))
mean_Y = np.mean(train_Y_aug[:, 0])
std_Y = np.std(train_Y_aug[:, 0])
train_Y_aug[:, 0]  = (train_Y_aug[:, 0] - mean_Y) / std_Y  # standardize

def optimize_model_with_scipy(model,maxiter=1000, disp=50):
    optimizer = gpflow.optimizers.Scipy()
    optimizer.minimize(
        model.training_loss,
        variables=model.trainable_variables,
        method="l-bfgs-b",
        options={"disp": disp, "maxiter": maxiter},
    )

base_kernel = Directional_Kernel(Rational_Quadratic, lengthscales=[60.0, 13.0], theta=0.10, variance=0.2, alpha=10.0)
coreg = gpflow.kernels.Coregion(output_dim=od, rank=2, active_dims=[2])
kernel = base_kernel * coreg
lik = gpflow.likelihoods.SwitchedLikelihood([gpflow.likelihoods.Gaussian(variance=0.2)]*od)
model = gpflow.models.VGP(kernel=kernel, likelihood=lik, data=(train_X_aug, train_Y_aug))
#% print_summary(model)
optimize_model_with_scipy(model, maxiter=1500, disp=50)


predicted_Y = []
for i in range(od):
    test_X = np.where(full[i] >= -10000)
    test_X = np.concatenate([x.reshape([-1, 1]) for x in test_X], axis=1)
    test_X = test_X.astype(np.float64)
    test_X = np.hstack((test_X, i*np.ones((test_X.shape[0], 1))))
    predicted_Y.append(model.predict_f(test_X, full_cov=False)[0].numpy().reshape(full[i].shape) * std_Y + mean_Y)

predicted_Y2 = predicted_Y.copy()  # The prediction with observed location unchanged
for i in range(od):
    predicted_Y2[i][mask[i]] = data[i][mask[i]]
    test_mae2 = mae(predicted_Y2[i], full[i])
    test_rmse2 = rmse(predicted_Y2[i], full[i])
    print(f'Lane{i}, test mae2: {test_mae2:.3f}, rmse2: {test_rmse2:.3f}.')
    plt.matshow(predicted_Y2[i], cmap='jet_r')
    

# print(f'{test_name}. theta: {model.kernel.theta.numpy():.3f}, n:{n},'
#       f'lengthscale: {model.kernel.lengthscale.numpy()}, likelihood: {model.likelihood.variance.numpy()}, time: {time.time()-time0:.3f}')
# print(f'Train mae: {train_mae:.3f}, rmse: {train_rmse:.3f}. Test mae: {test_mae:.3f}, rmse: {test_rmse:.3f}. '
#       f'Test mae2: {test_mae2:.3f}, rmse2: {test_rmse2:.3f}.')

# Save the model
save_pickle(gpflow.utilities.parameter_dict(model), f'./model.pkl')
savemat('./predict.mat',{'y3':predicted_Y2[0], 'y4':predicted_Y2[1]},do_compression=True)